In [106]:
using Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
using SatellitePlayground
using SatelliteDynamics
using Plots
using LinearAlgebra
SP = SatellitePlayground

include("nominal_dynamics.jl")
include("rollout.jl")

  Activating project at `~/Downloads/cmu_classes/16745/final_project`


generate_jacobians (generic function with 1 method)

In [107]:
J = SP.default_parameters.J
x0 = SP.initialize_orbit()

steps = 100
dt = 0.1
(states, times) = rollout(x0, steps, dt)
ref_traj = [
    SP.RBState(
        attitude=x0.attitude,
        angular_velocity=x0.angular_velocity,
        position=state.position,
        velocity=state.velocity
    )
    for state in states
    
]
@time As, Bs = generate_jacobians(ref_traj, times, J)

intialized orbit!


[100/100]: norm(ω)=1.140 r=<6712693.416 -0.002 -78867.541> b=<-13.527 6.442 18.259> dt=0.100
Simulation complete!


  6.778260 seconds (30.42 M allocations: 1.465 GiB, 6.33% gc time, 99.71% compilation time)


(Any[[1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0186495509297933 -0.18473916711140503 … 0.9596914936929323 0.1400887069031812; -0.07196844226211768 -0.10776226369391065 … -0.1400887069031812 0.9596914936929323], [1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0186495509297933 -0.18473916711140503 … 0.9596914936929323 0.1400887069031812; -0.07196844226211768 -0.10776226369391065 … -0.1400887069031812 0.9596914936929323], [1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0186495509297933 -0.18473916711140503 … 0.9596914936929323 0.1400887069031812; -0.07196844226211768 -0.10776226369391065 … -0.1400887069031812 0.9596914936929323], [1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0186495509297933 -0.18473916711140503 … 0.9596914936929323 0.1400887069031812; -0.07196844226211768 -0.10776226369391065 … -0.1400887069031812 0.9596914936929323], [1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0186495509297933 -0.18473916711140503 … 0.9596914936929323 0.1400887069031812; -0.07196844226211768 -0.10776226369

In [108]:
function hat(ω)
    # TODO: implement the hat function 
    ω̂ = [0 -ω[3] ω[2]
        ω[3] 0 -ω[1]
        -ω[2] ω[1] 0]
    
    return ω̂
end

function L(q)
    # TODO: implement L 
    #_L = zeros(4,4)
    _L = [q[1] -q[2:end]'
        q[2:end] q[1]*I + hat(q[2:end])]
    return _L
end

function R(q)
    # TODO: implement R 
    #_R = zeros(4,4)
    _R = [q[1] -q[2:end]'
        q[2:end] q[1]*I - hat(q[2:end])]
    return _R
end

# TODO: implement H function 
H = zeros(4,3)
H = [0 0 0
    1 0 0
    0 1 0
    0 0 1]

function G(q)
    q_mat = [-q[2] q[1] q[4] -q[3];
             -q[3] -q[4] q[1] q[2];
            -q[4] q[3] -q[2] q[1]]
    _G = [I zeros(3,4);
            zeros(3,3) q_mat]
    return _G
end

G (generic function with 1 method)

In [117]:
N = length(ref_traj) 
x0 = ref_traj[1]

# TODO: use TVLQR to generate K's 

# use this for TVLQR tracking cost 
Q_lqr = diagm([0.1,0.1,0.1, 1,1,1])
Qf_lqr = 10*Q_lqr
R_lqr = 0.01*diagm(ones(3))
nu = 3
nx = 13

K = [zeros(nu,nx) for i=1:N-1]
S = [zeros(nx,nx) for i = 1:N]
S[N]=deepcopy(Qf_lqr)
for j = N-1:-1:1
    xk = ref_traj[j]
    qk = xk.attitude
    Gk = G(qk)
    A = Gk*As[j]*Gk'
    B = Gk*Bs[j]
    K[j] = (R_lqr + B'S[j+1] * B) \ (B'S[j+1] * A)
    S[j] = Q_lqr + A'S[j+1] * (A - B * K[j])
    display(S[j])
end

6×6 Matrix{Float64}:
 0.205844   0.321256   0.145725   0.306789  0.569091   0.0517975
 0.321256   1.2005     0.635827   0.487847  1.60952    0.993388
 0.145725   0.635827   0.621488  -0.302397  0.534551   1.49555
 0.306789   0.487847  -0.302397   3.82828   3.26285   -2.3078
 0.569091   1.60952    0.534551   3.26285   8.25623    1.77465
 0.0517975  0.993388   1.49555   -2.3078    1.77465    9.26674

6×6 Matrix{Float64}:
 0.278325  0.63428   0.399292  0.144765  0.747178  0.703555
 0.63428   2.40255   1.47863   0.386569  2.69562   2.76057
 0.399292  1.47863   1.08174   0.143515  1.70207   2.00276
 0.144765  0.386569  0.143515  1.65329   1.08702   0.106358
 0.747178  2.69562   1.70207   1.08702   6.21313   4.42725
 0.703555  2.76057   2.00276   0.106358  4.42725   6.92438

6×6 Matrix{Float64}:
 0.398932  1.0785    0.689485   0.113632   0.945645   1.04922
 1.0785    4.03242   2.53731    0.294373   3.43232    3.98795
 0.689485  2.53731   1.76349    0.103742   2.18397    2.75663
 0.113632  0.294373  0.103742   1.50105    0.719771  -0.0343837
 0.945645  3.43232   2.18397    0.719771   5.43672    4.23578
 1.04922   3.98795   2.75663   -0.0343837  4.23578    6.99413

6×6 Matrix{Float64}:
 0.516712   1.50248   0.955708    0.0854861  1.0168     1.21208
 1.50248    5.55715   3.49312     0.193376   3.67072    4.549
 0.955708   3.49312   2.36089     0.0398038  2.31122    3.07924
 0.0854861  0.193376  0.0398038   1.47268    0.572632  -0.168539
 1.0168     3.67072   2.31122     0.572632   4.75556    3.64803
 1.21208    4.549     3.07924    -0.168539   3.64803    6.50196

6×6 Matrix{Float64}:
 0.620724   1.8709    1.18032      0.0688385   1.02633    1.26333
 1.8709     6.86111   4.2868       0.132893    3.68417    4.70613
 1.18032    4.2868    2.84246      0.00101147  2.29473    3.14554
 0.0688385  0.132893  0.00101147   1.46808     0.522041  -0.226555
 1.02633    3.68417   2.29473      0.522041    4.29051    3.12898
 1.26333    4.70613   3.14554     -0.226555    3.12898    5.9249

...

6×6 Matrix{Float64}:
  7.34375      27.4368    17.0356    -0.00269075  0.886922   1.16319
 27.4368      104.078     64.5918    -0.147013    3.42314    4.68898
 17.0356       64.5918    40.2552    -0.179028    2.166      3.17459
 -0.00269075   -0.147013  -0.179028   1.48779     0.430815  -0.355288
  0.886922      3.42314    2.166      0.430815    3.5461     2.20037
  1.16319       4.68898    3.17459   -0.355288    2.20037    4.70415

6×6 Matrix{Float64}:
  7.40493      27.6949    17.203     -0.00327748  0.885673   1.16207
 27.6949      105.155     65.2882    -0.149611    3.42178    4.68962
 17.203        65.2882    40.7046    -0.180778    2.16609    3.17627
 -0.00327748   -0.149611  -0.180778   1.48806     0.429987  -0.356492
  0.885673      3.42178    2.16609    0.429987    3.54584    2.20059
  1.16207       4.68962    3.17627   -0.356492    2.20059    4.70444

6×6 Matrix{Float64}:
  7.46583      27.9524    17.3703    -0.00386292  0.884423   1.16095
 27.9524      106.233     65.9852    -0.152209    3.42041    4.69027
 17.3703       65.9852    41.1547    -0.182528    2.16618    3.17796
 -0.00386292   -0.152209  -0.182528   1.48834     0.42916   -0.357695
  0.884423      3.42041    2.16618    0.42916     3.54558    2.20081
  1.16095       4.69027    3.17796   -0.357695    2.20081    4.70473

6×6 Matrix{Float64}:
  7.52644      28.2093    17.5373    -0.00444703  0.883174   1.15983
 28.2093      107.31      66.6827    -0.154807    3.41904    4.69091
 17.5373       66.6827    41.6057    -0.184281    2.16627    3.17965
 -0.00444703   -0.154807  -0.184281   1.48862     0.428332  -0.358898
  0.883174      3.41904    2.16627    0.428332    3.54533    2.20103
  1.15983       4.69091    3.17965   -0.358898    2.20103    4.70503

6×6 Matrix{Float64}:
  7.58676      28.4656    17.7041    -0.00502984  0.881925   1.15871
 28.4656      108.387     67.3809    -0.157406    3.41767    4.69155
 17.7041       67.3809    42.0573    -0.186035    2.16636    3.18133
 -0.00502984   -0.157406  -0.186035   1.4889      0.427505  -0.360102
  0.881925      3.41767    2.16636    0.427505    3.54507    2.20124
  1.15871       4.69155    3.18133   -0.360102    2.20124    4.70532

In [118]:

# here are the real parameters (different than the one we used for DIRCOL)
# this model mismatch is what's going to require the TVLQR controller to track
# the trajectory successfully. 
J = SP.default_parameters.J
x0_2 = SP.initialize_orbit()

steps = 100
dt = 0.1
(states, times) = rollout(x0_2, steps, dt)
Xsim = [
    SP.RBState(
        attitude=x0_2.attitude,
        angular_velocity=x0_2.angular_velocity,
        position=state.position,
        velocity=state.velocity
    )
    for state in states
    
]
Usim = [zeros(3) for i=1:steps]

# TODO: simulate closed loop system 
for i = 1:(N-1)
    # TODO: add feeback control (right now it's just feedforward)
    traj_step = ref_traj[i]
    xsimi = Xsim[i]
    xk_ref = traj_step.position
    vk_ref = traj_step.velocity
    qk_ref = traj_step.attitude
    ωk_ref = traj_step.angular_velocity
    xk_simi = xsimi.position
    vk_simi = xsimi.velocity
    qk_simi = xsimi.attitude
    ωk_simi = xsimi.angular_velocity
    e1 = qk_ref[1]*qk_simi[1] + qk_ref[2:4]'*qk_simi[2:4]
    e24 = qk_ref[1]*qk_simi[2:4] - qk_simi[1]*qk_ref[2:4] - cross(qk_ref[2:4],qk_simi[2:4])
    ϕ = e24/(1+e1)
    δxk = [ωk_simi - ωk_ref; 2*ϕ]
    Usim[i] = clamp.(-K[i]*δxk, -10, 10)
end

intialized orbit!
[100/100]: norm(ω)=0.592 r=<-84651.924 0.000 -6488664.500> b=<-28.548 -25.833 34.251> dt=0.100
Simulation complete!


In [115]:
Xsim[N].attitude

4-element StaticArraysCore.SVector{4, Float64} with indices SOneTo(4):
 -0.12104431602759198
  0.990828841941859
  0.04652631467086182
 -0.037970799000217485